# Contact-based Object Classification

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from utils.dataloader import ContactDataset  # Import our custom dataset class

In [2]:
from utils.dataloader import CATEGORIES
print(CATEGORIES)

{'Blueball': 0, 'Box': 1, 'Pencilcase': 2, 'Pinkball': 3, 'StuffedAnimal': 4, 'Tennis': 5, 'Waterbottle': 6}


In [3]:
# Define the classification model
class ContactClassifier(nn.Module):
    def __init__(self, input_size=8, hidden_size=16, num_classes=len(CATEGORIES)):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )
        
    def forward(self, x):
        return self.network(x)

In [4]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model = model.to(device)
    best_val_acc = 0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
        # Validation phase
        model.eval()
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                preds = outputs.argmax(dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_labels, val_preds)
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation Accuracy: {val_acc:.4f}')
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            
    return model

In [5]:
# Setup
DATA_DIR = "data/IDL_Project"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create dataset
dataset = ContactDataset(data_dir=DATA_DIR, labels=CATEGORIES)
print(len(dataset))

# Split dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(len(train_dataset), len(val_dataset))
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)

37
29 8


In [6]:
# Main training pipeline
# Initialize model, criterion, and optimizer
model = ContactClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

# Evaluate on validation set
model.eval()
val_preds = []
val_labels = []

with torch.no_grad():
    for features, labels in val_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        preds = outputs.argmax(dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# Print classification report
print("\nClassification Report:")
print(classification_report(val_labels, val_preds, 
                            target_names=list(CATEGORIES.keys())))

Epoch 1/50
Training Loss: 4.0832
Validation Accuracy: 0.2500
Epoch 2/50
Training Loss: 2.2968
Validation Accuracy: 0.0000
Epoch 3/50
Training Loss: 2.0562
Validation Accuracy: 0.0000
Epoch 4/50
Training Loss: 2.0027
Validation Accuracy: 0.0000
Epoch 5/50
Training Loss: 1.9848
Validation Accuracy: 0.0000
Epoch 6/50
Training Loss: 1.9762
Validation Accuracy: 0.0000
Epoch 7/50
Training Loss: 1.9532
Validation Accuracy: 0.0000
Epoch 8/50
Training Loss: 1.9812
Validation Accuracy: 0.0000
Epoch 9/50
Training Loss: 1.9858
Validation Accuracy: 0.0000
Epoch 10/50
Training Loss: 1.9526
Validation Accuracy: 0.0000
Epoch 11/50
Training Loss: 1.9519
Validation Accuracy: 0.0000
Epoch 12/50
Training Loss: 1.9114
Validation Accuracy: 0.0000
Epoch 13/50
Training Loss: 1.9037
Validation Accuracy: 0.0000
Epoch 14/50
Training Loss: 1.9370
Validation Accuracy: 0.0000
Epoch 15/50
Training Loss: 1.9253
Validation Accuracy: 0.0000
Epoch 16/50
Training Loss: 1.9031
Validation Accuracy: 0.0000
Epoch 17/50
Train

/home/xly/mambaforge/envs/IDL/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xly/mambaforge/envs/IDL/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xly/mambaforge/envs/IDL/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/xly/mambafor